In [1]:
from class_definitions import *

In [2]:
Wines = []
for (dirpath,dirnames,filenames) in walk('Wines/'):
    for filename in filenames:
        pkl = joblib.load(dirpath+'/'+filename)
        Wines.append(pkl)

In [3]:
len(Wines)

237

In [13]:
vivino_wines = []
for w in Wines:
    vivino_wines.extend(w.vivino)

In [22]:
vivino_ids = [vw['wine_link'].split('/')[-1] for vw in vivino_wines]
vivino_ids[:10]

['4195785',
 '3409016',
 '4927533',
 '3527904',
 '5083637',
 '4470482',
 '1611904',
 '141749720',
 '18344674',
 '78074287']

### WineSearcher Errors 

In [16]:
[w for w in Wines if w.wineSearcher == '403']

[Wine: Jacquère Cuvée Baujues 2016 St. Germain, Source: name-search,
 Wine: Bugey Cerdon Rosé Demi-Sec NV Dentelle, Source: name-search,
 Wine: Plavac Mali "Marica" 2017 Mrgudic, Source: name-search,
 Wine: Riesling Fass 2 Extra Trocken 2018 Lauer, Source: name-search,
 Wine: Muscadet Sèvre-et-Maine 2018 Batard Langelier, Source: name-search,
 Wine: C-d-P Tradition 750ml 2016 Bois de Boursan, Source: name-search,
 Wine: Silvaner Vulkan 2018 Helmut Dolde, Source: name-search,
 Wine: Malvasia "Bela" 2014 Rodica, Source: name-search,
 Wine: Naturalmente Frizzante Bianco 2018 Casa Belfi, Source: name-search,
 Wine: 2017 spatburgunder kabinett koehler ruprecht, Source: name-search,
 Wine: Sauvignon Blanc 2019 Le Petit Godon, Source: name-search,
 Wine: Sauvignon Blanc Pouch 1.5L Maivino, Source: name-search,
 Wine: Cabernet Sauvignon "Corrupto" 2019 Laurent, Source: name-search,
 Wine: Sincronia 2018 Mesquida Mora, Source: name-search,
 Wine: Vin de Savoie 2018 Jean Vullien, Source: name-se

In [5]:
kcw_wines = pd.read_csv('kings_county_wines.csv',index_col=0)

wine_names = list(kcw_wines['name'])

wineSearcher = pd.read_csv('wine-searcher_data.csv')

In [6]:
matched = [x for x in wine_names if x in wineSearcher['name'].to_list()]

In [8]:
wineSearcherData = pd.DataFrame.from_records([wine.wineSearcher for wine in Wines if type(wine.wineSearcher)==dict])
wineSearcherData.dropna(axis=0,thresh=3)

,url,name,Producer,Region/Appellation,Grape/Blend,Food Suggestion,Wine Style,Alcohol Content,Notes
1,NaN,Truffle Hound 2017 Unico Zelo,Unico Zelo,Clare Valley,Rare Red Blend,Beef and Venison,Red - Rich and Intense,12 - 13.5%,Indicative blend: Barbera and Nebbiolo
2,NaN,Sforzato di Valtellina 2013 Alfio Mozzi,Azienda Agricola Alfio Mozzi,Sforzato di Valtellina,Nebbiolo,Beef and Venison,Red - Rich and Intense,NaN,NaN
3,NaN,Cabernet Sauvignon 2017 Oracle Cellars,NaN,Mendocino,Cabernet Sauvignon,Beef and Venison,Red - Bold and Structured,NaN,NaN
5,NaN,Sauvignon Blanc 2019 Tablelands,TL Wine Co,New Zealand,Sauvignon Blanc,Goats' Cheese and Feta,White - Green and Flinty,12.5 - 14%,NaN
7,NaN,Zinfandel 2017 The People's Wine Co.,The People's Wine Company,California,Zinfandel,Lamb,Red - Rich and Intense,NaN,NaN
...,...,...,...,...,...,...,...,...,...
205,NaN,Sciaccarelu '1769' 2015 Clos Venturi,Domaine Vico,Vin de Corse,Sciaccarello (Mammolo),NaN,NaN,13.5%,NaN
206,NaN,"Soave Classico ""Otto"" 2018 Prá",Azienda Agricola Pra,Soave Classico,Soave Blend,White Fish,White - Green and Flinty,12%,Organic wine.
207,NaN,Tai Rosso 2018 Rezzadore,Azienda Agricola Rezzadore,Colli Berici,Grenache (Garnacha),Beef and Venison,Red - Rich and Intense,NaN,NaN
208,NaN,Merlot 2017 Je Déteste Merlot,Les Vignerons de Roquemaure,IGP Gard,Merlot,NaN,NaN,NaN,NaN


No result for wine-searcher, but one result for vivino. 
Re-instantiate from the vivino link for wine-searcher match?

In [9]:
wineSearcher_noResults = []
for W in Wines:
    if W.wineSearcher != '403':
        if len(W.wineSearcher.keys())==2:
            wineSearcher_noResults.append(W)

In [10]:
[(W.name,len(W.vivino)) for W in wineSearcher_noResults]

[('Zweigelt 2016 Bauer', 20),
 ('Spatburgunder Kabinett 2017 Koehler-Ruprecht', 1),
 ('Costa delle Sesia Uvaggio 2015 Sperino', 1),
 ('Aloxe Corton "Les Citernes" 2015 Bidault', 1),
 ('Refošk Baracija 2017 Clai Wines', 20),
 ('Lambrusco Bollarossa NV Corte Manzini', 1),
 ("Imereti 2018 Amiran's Wine Cellar", 20),
 ('Cinsault 2017 Turbigo', 1),
 ('Beerenauslese 500mL 2015 Proidl', 2),
 ('Autour du Pigeonnier 2016 Benoît Roseau', 1),
 ('Garnacha "Campo de Borja" 2018 Dos Molinos', 1),
 ('Cab Sauv "Super Cali" Vogelzang 2013 CCGP', 20),
 ('Saint-Amour À La Folie 2017 Arpaye', 1),
 ('Piccolo Derthona 2017 Terralba', 1),
 ('Pet-Nat Orbis Moderandi 2019 OCD', 20),
 ('Fixin "Les Champstions" 2016 Domaine Collotte', 1),
 ("Catarratto L'Isola del Satiro 2018 Alcesti", 1),
 ("Sylvaner d'Alsace 2017 Vincent Fleith", 1),
 ('Riesling Kaefferkopf 1.5L 2016 Schoech', 2),
 ('Pinot Noir "La Belouse" 2017 L\'Echelette', 1),
 ('Langhe Nebbiolo 2017 Fratelli', 17),
 ('Grüner Veltliner Kremstal 2017 Proidl

In [11]:
easy_vivino_matches = []
for W in Wines:
    if len(W.vivino)==1:
        easy_vivino_matches.extend(W.vivino)

len(easy_vivino_matches)

94

### Scrape and store Vivino

In [25]:
import json 

In [26]:
def scrape_vivino(wine_id):
    url = f"https://vivino.com/wines/{str(wine_id)}"
    page = requests.get(url,headers=Wine.headers)
    bs = BeautifulSoup(page.content)
    
    ld_json = bs.find('script',attrs={'type':"application/ld+json"})
    overview = json.loads(ld_json.text)['mainEntity']
    
    scripts = bs.find_all('script')
    
    for s in scripts:
        if "winePageInformation" in s.text:
            info = s
    a = info.text.split(';')[1]
    b = a.strip('\n  window.__PRELOADED_STATE__.winePageInformation = ')
    
    wine_info = json.loads(b)
    
    return wine_info

In [27]:
result = scrape_vivino(vivino_ids[10])

In [81]:
result['vintage']['wine']

{'id': 2425918,
 'name': 'Zweigelt Reserve Haidberg',
 'seo_name': 'zweigelt-reserve-haidberg',
 'type_id': 1,
 'is_natural': False,
 'region': {'id': 888,
  'name': 'Weinviertel',
  'name_en': '',
  'seo_name': 'weinviertel',
  'country': {'code': 'at',
   'name': 'Austria',
   'native_name': 'Österreich',
   'seo_name': 'austria',
   'currency': {'code': 'EUR', 'name': 'Euros', 'prefix': '€', 'suffix': None},
   'regions_count': 24,
   'users_count': 147508,
   'wines_count': 43242,
   'wineries_count': 4240,
   'most_used_grapes': [{'id': 233,
     'name': 'Grüner Veltliner',
     'seo_name': 'gruner-veltliner',
     'has_detailed_info': False,
     'wines_count': 28092},
    {'id': 257,
     'name': 'Zweigelt',
     'seo_name': 'zweigelt',
     'has_detailed_info': False,
     'wines_count': 23104},
    {'id': 15,
     'name': 'Riesling',
     'seo_name': 'riesling',
     'has_detailed_info': True,
     'wines_count': 202743}]},
  'background_image': None,
  'class': {'typecast_map

In [78]:
result['wine'].keys()

dict_keys(['id', 'name', 'seo_name', 'type_id', 'is_natural', 'region', 'review_status', 'winery', 'image', 'statistics', 'rank', 'hidden', 'vintages', 'style'])

In [75]:
result['vintage']['wine']['region']

{'id': 888,
 'name': 'Weinviertel',
 'name_en': '',
 'seo_name': 'weinviertel',
 'country': {'code': 'at',
  'name': 'Austria',
  'native_name': 'Österreich',
  'seo_name': 'austria',
  'currency': {'code': 'EUR', 'name': 'Euros', 'prefix': '€', 'suffix': None},
  'regions_count': 24,
  'users_count': 147508,
  'wines_count': 43242,
  'wineries_count': 4240,
  'most_used_grapes': [{'id': 233,
    'name': 'Grüner Veltliner',
    'seo_name': 'gruner-veltliner',
    'has_detailed_info': False,
    'wines_count': 28092},
   {'id': 257,
    'name': 'Zweigelt',
    'seo_name': 'zweigelt',
    'has_detailed_info': False,
    'wines_count': 23104},
   {'id': 15,
    'name': 'Riesling',
    'seo_name': 'riesling',
    'has_detailed_info': True,
    'wines_count': 202743}]},
 'background_image': None,
 'class': {'typecast_map': {'background_image': {}, 'class': {}}}}

In [73]:
result['vintage']['wine']

{'id': 2425918,
 'name': 'Zweigelt Reserve Haidberg',
 'seo_name': 'zweigelt-reserve-haidberg',
 'type_id': 1,
 'is_natural': False,
 'region': {'id': 888,
  'name': 'Weinviertel',
  'name_en': '',
  'seo_name': 'weinviertel',
  'country': {'code': 'at',
   'name': 'Austria',
   'native_name': 'Österreich',
   'seo_name': 'austria',
   'currency': {'code': 'EUR', 'name': 'Euros', 'prefix': '€', 'suffix': None},
   'regions_count': 24,
   'users_count': 147508,
   'wines_count': 43242,
   'wineries_count': 4240,
   'most_used_grapes': [{'id': 233,
     'name': 'Grüner Veltliner',
     'seo_name': 'gruner-veltliner',
     'has_detailed_info': False,
     'wines_count': 28092},
    {'id': 257,
     'name': 'Zweigelt',
     'seo_name': 'zweigelt',
     'has_detailed_info': False,
     'wines_count': 23104},
    {'id': 15,
     'name': 'Riesling',
     'seo_name': 'riesling',
     'has_detailed_info': True,
     'wines_count': 202743}]},
  'background_image': None,
  'class': {'typecast_map

In [63]:
result['wine']['style']

In [64]:
result['vintage']['wine']['style']

{'id': 255,
 'seo_name': 'austrian-zweigelt',
 'regional_name': 'Austrian',
 'varietal_name': 'Zweigelt',
 'name': 'Austrian Zweigelt',
 'image': None,
 'background_image': None,
 'description': 'The fruit-forward and spicy Zweigelt, with flavors of sour cherry, raspberry, cinnamon, violets and pepper, has soft tannins and is the most widely planted red grape in Austria. It can taste similar to a spicy Beaujolais, a racy Pinot Noir or an inky California Zinfandel. \n\nIn 1922, Zweigelt was created by researcher Frank Zweigelt, by crossing an Austrian grape called Blaufränkish, which gives the wine its acidity, with St. Laurent, which imparts bright cherry flavors and a smooth texture.\n\nZweigelt is planted throughout Austria, with the most expressive wines coming from the Neusiedlersee DAC of Burgenland, in the easternmost part of country.\n\nWhile most Zweigelt styles are dry, the grape is also used to make sweeter styles such as strohwein and ice wine.',
 'blurb': None,
 'interestin

In [56]:
result['vintage']

{'id': 10606567,
 'seo_name': 'norbert-bauer-zweigelt-reserve-haidberg-uv',
 'name': 'Norbert Bauer Zweigelt Reserve Haidberg',
 'statistics': {'status': 'Normal',
  'ratings_count': 15,
  'ratings_average': 3.7,
  'labels_count': 9,
  'reviews_count': 2},
 'image': {'location': '//images.vivino.com/labels/WEglRvsHR3qedxic0lSp-w.jpg',
  'variations': {'large': '//images.vivino.com/thumbs/WEglRvsHR3qedxic0lSp-w_375x500.jpg',
   'medium': '//images.vivino.com/thumbs/WEglRvsHR3qedxic0lSp-w_150x200.jpg',
   'medium_square': '//images.vivino.com/thumbs/WEglRvsHR3qedxic0lSp-w_150x150.jpg',
   'small_square': '//images.vivino.com/thumbs/WEglRvsHR3qedxic0lSp-w_80x80.jpg'}},
 'wine': {'id': 2425918,
  'name': 'Zweigelt Reserve Haidberg',
  'seo_name': 'zweigelt-reserve-haidberg',
  'type_id': 1,
  'is_natural': False,
  'region': {'id': 888,
   'name': 'Weinviertel',
   'name_en': '',
   'seo_name': 'weinviertel',
   'country': {'code': 'at',
    'name': 'Austria',
    'native_name': 'Österreic

### selenium scraper 

In [285]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [502]:
chrome_driver_path = "/Users/schlinkertc/code/sm_selenium/chromedriver"
driver = webdriver.Chrome(executable_path=chrome_driver_path)

In [503]:
driver.get(url)

In [504]:
master_wine_page = driver.find_element_by_id('master-wine-page-app')
driver.set_window_size(width=742,height=983)

a_tags = driver.find_elements_by_tag_name('a')

for a in a_tags:
    a_class = a.get_attribute('class')
    if 'toggleShowAll' in a_class:
        show_all = a
        break

a.click()


In [506]:
text = master_wine_page.text

In [508]:
[x for x in text.split('\n') if 'mentions of' in x]

['8 mentions of black fruit notes',
 '6 mentions of red fruit notes',
 '6 mentions of oaky notes',
 '6 mentions of earthy notes',
 '5 mentions of spices notes',
 '2 mentions of microbio notes',
 '1 mentions of floral notes',
 '1 mentions of vegetal notes',
 '1 mentions of dried fruit notes']

In [491]:
driver.quit()

In [500]:
import time
def selenium_scrape(wine_id):
    url = f"https://vivino.com/wines/{str(wine_id)}"
    
    chrome_driver_path = "/Users/schlinkertc/code/sm_selenium/chromedriver"
    driver = webdriver.Chrome(executable_path=chrome_driver_path)
    
    driver.get(url)
    
    master_wine_page = driver.find_element_by_id('master-wine-page-app')
    
    driver.set_window_size(width=742,height=983)

    svgs = master_wine_page.find_elements_by_tag_name('svg')
    [s.get_attribute('class') for s in svgs]

    tasteNotes = []
    for s in svgs:
        try:
            svg_class = s.get_attribute('class')
            if 'tasteNote' in svg_class:
                tasteNotes.append(s)
        except:
            continue 
            
    a_tags = driver.find_elements_by_tag_name('a')

    for a in a_tags:
        a_class = a.get_attribute('class')
        if 'toggleShowAll' in a_class:
            show_all = a
            break

    a.click()

    note_texts = []
    for i in range(len(tasteNotes)):
        tasteNotes[i].click()
        divs = driver.find_elements_by_tag_name('div')
        note_text = divs[1].text
        note_texts.append(note_text.split('\n'))
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        time.sleep(3)
        
    spans = [x for x in master_wine_page.find_elements_by_tag_name('span')]

    styles = []
    for span in spans:
        try:
            style = span.get_attribute('style')
            if style.startswith('width'):
                styles.append(style)
        except:
            continue
    driver.quit()
        
    return note_texts,styles

In [529]:
# pop up doesn't hold long enough to srape the right text. likely not worth it
#note_texts, styles = selenium_scrape(158494162)

In [3]:
Wines

[Wine: Zweigelt 2016 Bauer, Source: name-search,
 Wine: Truffle Hound 2017 Unico Zelo, Source: name-search,
 Wine: Sforzato di Valtellina 2013 Alfio Mozzi, Source: name-search,
 Wine: Cabernet Sauvignon 2017 Oracle Cellars, Source: name-search,
 Wine: Spatburgunder Kabinett 2017 Koehler-Ruprecht, Source: name-search,
 Wine: Sauvignon Blanc 2019 Tablelands, Source: name-search,
 Wine: Costa delle Sesia Uvaggio 2015 Sperino, Source: name-search,
 Wine: Zinfandel 2017 The People's Wine Co., Source: name-search,
 Wine: Aloxe Corton "Les Citernes" 2015 Bidault, Source: name-search,
 Wine: Moschofilero Fteri 2017 Troupis Winery, Source: name-search,
 Wine: Collio Ribolla Gialla Oslavia 2018 Fiegl, Source: name-search,
 Wine: Beaujolais-Villages Le Bouteau 2016 Granger, Source: name-search,
 Wine: Gruner Veltliner 1L 2019 Hugl Weine, Source: name-search,
 Wine: Rioja 2014 Sierra de la Demanda, Source: name-search,
 Wine: Mencia "Fusco" 2018 Bodegas Albamar, Source: name-search,
 Wine: Pinot G

In [532]:
Wine.WINES

237